<a href="https://colab.research.google.com/github/ayachiii/soturon/blob/main/export_rr_interval%E8%87%AA%E5%88%86%E3%81%A7%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# (セル 1)
# 必要なライブラリをインストール
!pip install heartpy pandas matplotlib

In [36]:
# (セル 2)
from google.colab import files

print("ECG_人名_フェーズ.csv をアップロードしてください：")
uploaded = files.upload()

# アップロードされたファイル名を取得（複数ファイル非対応）
if len(uploaded) == 0:
    print("ファイルがアップロードされませんでした。")
else:
    INPUT_FILE = list(uploaded.keys())[0]
    print(f"\nファイル '{INPUT_FILE}' がアップロードされました。")

ECG_人名_フェーズ.csv をアップロードしてください：


Saving ECG_kumakura_flow_ultra.csv to ECG_kumakura_flow_ultra (1).csv

ファイル 'ECG_kumakura_flow_ultra (1).csv' がアップロードされました。


In [37]:
# (セル 3 - 修正版)
#
import numpy as np
import pandas as pd
import heartpy as hp
from matplotlib import pyplot as plt
import os

# --- 1. 設定項目 ---
# ★★★★★★★★★★★★★★★★★★★★★★★★
# ★★★ 修正点 ★★★
# サンプリング周波数を 500.0 (500Hz) に設定します。
# これで 800ms 前後の正しい値が出力されるはずです。
SAMPLING_RATE = 500.0
# ★★★★★★★★★★★★★★★★★★★★★★★★

# --- 2. 保存先の設定 (Colabの作業フォルダ) ---
# INPUT_FILE は ステップ2で定義済み
OUTPUT_DIR = "/content/"
output_file_name = INPUT_FILE.replace('ECG_', 'rr_interval_')
FULL_OUTPUT_PATH = os.path.join(OUTPUT_DIR, output_file_name)


# --- 3. 必要な関数 (ipynbファイルから) ---
# (今回はCSV保存には使用しませんが、.ipynbの定義に合わせて残します)
def replace_outliers_with_mean(df):
    mean_value = df.mean()
    std_dev = df.std()
    lower_bound = mean_value - 3 * std_dev
    upper_bound = mean_value + 3 * std_dev
    df_cleaned = df.where((df >= lower_bound) & (df <= upper_bound))
    df_cleaned = df_cleaned.fillna(mean_value)
    return df_cleaned

# --- 4. メイン処理 ---
def process_ecg_file():
    if 'INPUT_FILE' not in globals():
        print("エラー: ステップ2に戻ってファイルをアップロードしてください。")
        return

    print(f"\n処理を開始します: {INPUT_FILE}")

    # --- データの読み込み ---
    try:
        hrdata = hp.get_data(INPUT_FILE, delim=",", column_name="ECG")
    except Exception as e:
        print(f"ファイルの読み込み中にエラーが発生しました: {e}")
        return

    # --- RR間隔の計算 ---
    try:
        # 修正された SAMPLING_RATE (500.0) で処理を実行
        # (上下反転の -1 は削除しました)
        working_data, measures = hp.process(hrdata, SAMPLING_RATE,
                                            report_time=True, calc_freq=True)
    except Exception as e:
        print(f"HeartPyでの処理中にエラーが発生しました: {e}")
        return

    # .ipynb のロジックに合わせて、生のRRリストをDataFrameに変換
    rr_interval = pd.DataFrame(working_data["RR_list"], columns=['RR_Interval_ms'])

    # .ipynb のロジックに合わせて、外れ値処理を計算 (保存はしない)
    rr_interval_cleaned = replace_outliers_with_mean(rr_interval)

    # --- データの保存 ---
    try:
        # .ipynb と同じ 'rr_interval' (生のRR間隔) を保存します
        rr_interval.to_csv(FULL_OUTPUT_PATH, index=False)

        print("\n--- 処理完了 ---")
        print(f"ファイルがColab環境に保存されました。")
        print(f"保存先: {FULL_OUTPUT_PATH}")
        print("\n次のステップに進んで、ファイルの中身を確認・ダウンロードしてください。")

    except Exception as e:
        print(f"ファイルの保存中にエラーが発生しました: {e}")

# --- 5. スクリプトの実行 ---
process_ecg_file()


処理を開始します: ECG_kumakura_flow_ultra (1).csv

Finished in 0.137029 sec

--- 処理完了 ---
ファイルがColab環境に保存されました。
保存先: /content/rr_interval_kumakura_flow_ultra (1).csv

次のステップに進んで、ファイルの中身を確認・ダウンロードしてください。


/usr/local/lib/python3.12/dist-packages/heartpy/analysis.py:522: UserWarning: Short signal.

---------Warning:---------
too few peak-peak intervals for (reliable) frequency domain measure computation, frequency output measures are still computed but treat them with caution!

HF is usually computed over a minimum of 1 minute of good signal. LF is usually computed over a minimum of 2 minutes of good signal.VLF is usually computed over a minimum of 5 minutes of good signal.The LF/HF ratio is usually computed over minimum 24 hours, although an absolute minimum of 5 min has also been suggested.

For more info see: 
Shaffer, F., Ginsberg, J.P. (2017), An Overview of Heart Rate Variability Metrics and Norms.

Task Force of Pacing and Electrophysiology (1996), Heart Rate Variability, in: European Heart Journal, vol.17, issue 3, pp354-381

This warning will not repeat
  warnings.warn(msg, UserWarning)


In [38]:

try:
    if 'FULL_OUTPUT_PATH' in globals() and os.path.exists(FULL_OUTPUT_PATH):
        print(f"{FULL_OUTPUT_PATH} をダウンロードします...")
        files.download(FULL_OUTPUT_PATH)
    else:
        print("エラー: 保存されたRR間隔ファイルが見つかりません。ステップ3を再実行してください。")
except NameError:
     print("エラー: ステップ3が正常に実行されていません。")

/content/rr_interval_kumakura_flow_ultra (1).csv をダウンロードします...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>